In [1]:
import numpy as np
import pandas as pd
import os

mean social cost, unweighted or weighted (fixed)

In [51]:
weighted = True

res_path = 'outputs/results'

if weighted:
    # n_agents = [5]  # NOTE: only one at a time
    n_agents = [10]
    unweighted_tag = ''
    llm_unweighted_tag = ''
else:
    n_agents = [5]
    unweighted_tag = 'un'
    llm_unweighted_tag = '_unweighted'

n_locs = [1,2,3,4]
distributions = ['uniform', 'normal', 'beta1', 'beta2']
methods = ['PercentileRuleSystem','DictatorRuleSystem','ConstantRuleSystem','MoulinNetSystem','RegretNetSystem','NonSPRuleSystem']

n_trial, n_trial_evolop = 3, 3

for distribution in distributions:
    df = pd.read_csv(f'{res_path}/search_{distribution}_{unweighted_tag}weighted_rgt.csv')
    for n in n_agents:
        for k in n_locs:
            res_string = f'{distribution} & {k} & '
            for m in methods:
                value = df[((df['model'] == m) & (df['n'] == n) & (df['k'] == k) & (df['phase'] == 'test'))]['mean_social_cost'].item()
                if m == 'RegretNetSystem':
                    regret = df[((df['model'] == m) & (df['n'] == n) & (df['k'] == k) & (df['phase'] == 'test'))]['max_regret'].item()
                    regret_string = f' ({regret:.5f})' if regret > 0 else ''
                    res_string += f"{value:.5f}{regret_string} & "
                else:
                    res_string += f"{value:.5f} & "
            # LLM
            best_llm_sc, best_llm_rgt = np.full(n_trial, np.inf), np.full(n_trial, np.inf)
            best_llm_evolop_sc, best_llm_evolop_rgt = np.full(n_trial*n_trial_evolop, np.inf), np.full(n_trial*n_trial_evolop, np.inf)
            for j in range(n_trial):
                llm_res_path = f"../LLM-FL2/src/eoh/exp/{distribution}_{k}_{n}_{j+1}{llm_unweighted_tag}/results"
                if os.path.exists(llm_res_path) and os.path.exists(f"{llm_res_path}/top_1_performance.csv"):
                    df_llm = pd.read_csv(f"{llm_res_path}/top_1_performance.csv")
                    best_llm_sc[j] = df_llm[df_llm['Generation'] == 20]['Cost'].item()
                    best_llm_rgt[j] = df_llm[df_llm['Generation'] == 20]['Regret'].item()
                    for i in range(n_trial_evolop):
                        if os.path.exists(f"{llm_res_path}/top_1_performance_evolop_{i+1}.csv"):
                            df_llm_evolop = pd.read_csv(f"{llm_res_path}/top_1_performance_evolop_{i+1}.csv")
                            best_llm_evolop_sc[j*n_trial_evolop+i] = df_llm_evolop[df_llm_evolop['Generation'] == 20]['Cost'].item()
                            best_llm_evolop_rgt[j*n_trial_evolop+i] = df_llm_evolop[df_llm_evolop['Generation'] == 20]['Regret'].item()

            best_llm_id = np.argmin(best_llm_sc)
            best_llm_sc = best_llm_sc[best_llm_id]
            best_llm_rgt = best_llm_rgt[best_llm_id]
            regret_string = f' ({best_llm_rgt:.5f})' if best_llm_rgt > 0 else ''
            res_string += f"{best_llm_sc:.5f}{regret_string} & "

            best_llm_evolop_id = np.argmin(best_llm_evolop_sc)
            best_llm_evolop_sc = best_llm_evolop_sc[best_llm_evolop_id]
            best_llm_evolop_rgt = best_llm_evolop_rgt[best_llm_evolop_id]
            regret_string = f' ({best_llm_evolop_rgt:.5f})' if best_llm_evolop_rgt > 0 else ''
            res_string += f"{best_llm_evolop_sc:.5f}{regret_string}"

            res_string += ' \\\\' if k != n_locs[-1] else ' \\\\\\hline'
            print(res_string)

uniform & 1 & 0.22896 & 0.24250 & 0.25161 & 0.20888 & 0.21507 (0.00137) & 0.20888 & 0.20887 & 0.20887 \\
uniform & 2 & 0.09926 & 0.09106 & 0.12761 & 0.08175 & 0.08277 (0.00092) & 0.07285 & 0.08146 & 0.08146 \\
uniform & 3 & 0.05879 & 0.05695 & 0.08559 & 0.04455 & 0.04872 (0.00103) & 0.03521 & 0.04565 & 0.04429 \\
uniform & 4 & 0.04065 & 0.03866 & 0.06380 & 0.03439 & 0.03736 (0.00073) & 0.02001 & 0.03513 & 0.02794 (0.00096) \\\hline
normal & 1 & 0.09537 & 0.10546 & 0.11925 & 0.08795 & 0.09244 (0.00029) & 0.08791 & 0.08790 & 0.08790 \\
normal & 2 & 0.04909 & 0.04124 & 0.08153 & 0.03847 & 0.05340 (0.00060) & 0.03489 & 0.03739 (0.00049) & 0.03673 (0.00063) \\
normal & 3 & 0.03041 & 0.02729 & 0.04552 & 0.02485 & 0.02800 (0.00061) & 0.01779 & 0.01806 (0.00230) & 0.01779 (0.00184) \\
normal & 4 & 0.02051 & 0.01927 & 0.03512 & 0.01674 & 0.02252 (0.00044) & 0.01012 & 0.01600 & 0.01626 \\\hline
beta1 & 1 & 0.06260 & 0.06851 & 0.07157 & 0.05781 & 0.05848 (0.00069) & 0.05780 & 0.05779 & 0.05779 \\

max social cost (unweighted only)

In [54]:
res_path = 'outputs/results'

n_agents = [5]
llm_unweighted_tag = '_unweighted'

n_locs = [1,2,3,4]
distributions = ['uniform', 'normal', 'beta1', 'beta2']
methods = ['PercentileRuleSystem','DictatorRuleSystem','ConstantRuleSystem','MoulinNetSystem','RegretNetSystem']

n_trial, n_trial_evolop = 3, 3

for distribution in distributions:
    df = pd.read_csv(f'{res_path}/search_{distribution}_max_rgt.csv')
    for n in n_agents:
        for k in n_locs:
            res_string = f'{distribution} & {k} & '
            for m in methods:
                value = df[((df['model'] == m) & (df['n'] == n) & (df['k'] == k) & (df['phase'] == 'test'))]['max_social_cost'].item()
                if m == 'RegretNetSystem':
                    regret = df[((df['model'] == m) & (df['n'] == n) & (df['k'] == k) & (df['phase'] == 'test'))]['max_regret'].item()
                    regret_string = f' ({regret:.5f})' if regret > 0 else ''
                    res_string += f"{value:.5f}{regret_string} & "
                else:
                    res_string += f"{value:.5f} & "
            # LLM
            best_llm_sc, best_llm_rgt = np.full(n_trial, np.inf), np.full(n_trial, np.inf)
            best_llm_evolop_sc, best_llm_evolop_rgt = np.full(n_trial*n_trial_evolop, np.inf), np.full(n_trial*n_trial_evolop, np.inf)
            for j in range(n_trial):
                llm_res_path = f"../LLM-FL2/src/eoh/exp_max/{distribution}_{k}_{n}_{j+1}{llm_unweighted_tag}/results"
                if os.path.exists(llm_res_path) and os.path.exists(f"{llm_res_path}/top_1_performance.csv"):
                    df_llm = pd.read_csv(f"{llm_res_path}/top_1_performance.csv")
                    best_llm_sc[j] = df_llm[df_llm['Generation'] == 20]['Cost'].item()
                    best_llm_rgt[j] = df_llm[df_llm['Generation'] == 20]['Regret'].item()
                    for i in range(n_trial_evolop):
                        if os.path.exists(f"{llm_res_path}/top_1_performance_evolop_{i+1}.csv"):
                            df_llm_evolop = pd.read_csv(f"{llm_res_path}/top_1_performance_evolop_{i+1}.csv")
                            best_llm_evolop_sc[j*n_trial_evolop+i] = df_llm_evolop[df_llm_evolop['Generation'] == 20]['Cost'].item()
                            best_llm_evolop_rgt[j*n_trial_evolop+i] = df_llm_evolop[df_llm_evolop['Generation'] == 20]['Regret'].item()

            best_llm_id = np.argmin(best_llm_sc)
            best_llm_sc = best_llm_sc[best_llm_id]
            best_llm_rgt = best_llm_rgt[best_llm_id]
            regret_string = f' ({best_llm_rgt:.5f})' if best_llm_rgt > 0 else ''
            res_string += f"{best_llm_sc:.5f}{regret_string} & "

            best_llm_evolop_id = np.argmin(best_llm_evolop_sc)
            best_llm_evolop_sc = best_llm_evolop_sc[best_llm_evolop_id]
            best_llm_evolop_rgt = best_llm_evolop_rgt[best_llm_evolop_id]
            regret_string = f' ({best_llm_evolop_rgt:.5f})' if best_llm_evolop_rgt > 0 else ''
            res_string += f"{best_llm_evolop_sc:.5f}{regret_string}"

            res_string += ' \\\\' if k != n_locs[-1] else ' \\\\\\hline'
            print(res_string)

uniform & 1 & 0.45910 & 0.57267 & 0.43014 & 0.41278 & 0.40543 (0.00159) & 0.41438 & 0.41436 (0.00099) \\
uniform & 2 & 0.24176 & 0.34643 & 0.21752 & 0.18958 & 0.19008 (0.00349) & 0.20899 & 0.20899 \\
uniform & 3 & 0.12379 & 0.21548 & 0.14892 & 0.13713 & 0.12420 (0.00335) & 0.12378 & 0.11888 (0.00043) \\
uniform & 4 & 0.08241 & 0.11619 & 0.11608 & 0.08363 & 0.08916 (0.00366) & 0.04258 (0.00092) & 0.02128 (0.00137) \\\hline
normal & 1 & 0.22114 & 0.27568 & 0.23658 & 0.22104 & 0.21607 (0.00095) & 0.22143 (0.00014) & 0.22108 (0.00012) \\
normal & 2 & 0.12608 & 0.17095 & 0.14102 & 0.12982 & 0.10766 (0.01561) & 0.12607 & 0.12081 (0.00196) \\
normal & 3 & 0.06271 & 0.10851 & 0.10520 & 0.06277 & 0.06056 (0.00323) & 0.03215 (0.00376) & 0.02944 (0.00406) \\
normal & 4 & 0.03573 & 0.06290 & 0.08321 & 0.04096 & 0.04620 (0.00229) & 0.01476 (0.00161) & 0.01108 (0.00083) \\\hline
beta1 & 1 & 0.13590 & 0.16328 & 0.13531 & 0.12976 & 0.12503 (0.00156) & 0.14081 & 0.13064 (0.00015) \\
beta1 & 2 & 0.05839

arbitrary weights

In [66]:
res_path = 'outputs/results'

n_agents = [10]

n_locs = [4]
distributions = ['uniform']
methods = ['PercentileRuleSystem','DictatorRuleSystem','ConstantRuleSystem','MoulinNetSystem','RegretNetSystem','NonSPRuleSystem']

n_trial, n_trial_evolop = 3, 3
n_weight_set = 10

for distribution in distributions:
    df = pd.read_csv(f'{res_path}/search_{distribution}_arbi-weighted_rgt.csv')
    for n in n_agents:
        for k in n_locs:
            for w in range(n_weight_set):
                res_string = f'{w+1} & '
                for m in methods:
                    value = df[((df['model'] == m) & (df['n'] == n) & (df['k'] == k) & (df['phase'] == 'test'))]['mean_social_cost'].iloc[w]
                    if m == 'RegretNetSystem':
                        regret = df[((df['model'] == m) & (df['n'] == n) & (df['k'] == k) & (df['phase'] == 'test'))]['max_regret'].iloc[w]
                        regret_string = f' ({regret:.5f})' if regret > 0 else ''
                        res_string += f"{value:.5f}{regret_string} & "
                    else:
                        res_string += f"{value:.5f} & "
                # LLM
                best_llm_sc, best_llm_rgt = np.full(n_trial, np.inf), np.full(n_trial, np.inf)
                best_llm_evolop_sc, best_llm_evolop_rgt = np.full(n_trial*n_trial_evolop, np.inf), np.full(n_trial*n_trial_evolop, np.inf)
                for j in range(n_trial):
                    llm_res_path = f"../LLM-FL2/src/eoh/exp/{distribution}_{k}_{n}_{j+1}_w{w+1}/results"
                    if os.path.exists(llm_res_path) and os.path.exists(f"{llm_res_path}/top_1_performance.csv"):
                        df_llm = pd.read_csv(f"{llm_res_path}/top_1_performance.csv")
                        best_llm_sc[j] = df_llm[df_llm['Generation'] == 20]['Cost'].item()
                        best_llm_rgt[j] = df_llm[df_llm['Generation'] == 20]['Regret'].item()
                        for i in range(n_trial_evolop):
                            if os.path.exists(f"{llm_res_path}/top_1_performance_evolop_{i+1}.csv"):
                                df_llm_evolop = pd.read_csv(f"{llm_res_path}/top_1_performance_evolop_{i+1}.csv")
                                best_llm_evolop_sc[j*n_trial_evolop+i] = df_llm_evolop[df_llm_evolop['Generation'] == 20]['Cost'].item()
                                best_llm_evolop_rgt[j*n_trial_evolop+i] = df_llm_evolop[df_llm_evolop['Generation'] == 20]['Regret'].item()

                best_llm_id = np.argmin(best_llm_sc)
                best_llm_sc = best_llm_sc[best_llm_id]
                best_llm_rgt = best_llm_rgt[best_llm_id]
                regret_string = f' ({best_llm_rgt:.5f})' if best_llm_rgt > 0 else ''
                res_string += f"{best_llm_sc:.5f}{regret_string} & "

                best_llm_evolop_id = np.argmin(best_llm_evolop_sc)
                best_llm_evolop_sc = best_llm_evolop_sc[best_llm_evolop_id]
                best_llm_evolop_rgt = best_llm_evolop_rgt[best_llm_evolop_id]
                regret_string = f' ({best_llm_evolop_rgt:.5f})' if best_llm_evolop_rgt > 0 else ''
                res_string += f"{best_llm_evolop_sc:.5f}{regret_string}"

                res_string += ' \\\\' if w != n_weight_set-1 else ' \\\\\\hline'
                print(res_string)

1 & 0.04069 & 0.04088 & 0.06449 & 0.03350 & 0.05797 (0.00189) & 0.02430 & 0.03168 & 0.03168 \\
2 & 0.04061 & 0.03931 & 0.06496 & 0.03347 & 0.05276 (0.00117) & 0.02358 & 0.03106 & 0.03106 \\
3 & 0.04068 & 0.04689 & 0.06376 & 0.03424 & 0.05572 (0.00144) & 0.02517 & 0.03085 & 0.03085 \\
4 & 0.04097 & 0.05787 & 0.06474 & 0.03632 & 0.05797 (0.00170) & 0.02561 & 0.03805 & 0.03235 \\
5 & 0.04121 & 0.06215 & 0.06389 & 0.04284 & 0.06016 (0.00190) & 0.02984 & 0.03805 & 0.03805 \\
6 & 0.04165 & 0.04279 & 0.06318 & 0.03964 & 0.05656 (0.00183) & 0.02337 & 0.03250 & 0.03250 \\
7 & 0.04083 & 0.05137 & 0.06426 & 0.03937 & 0.05837 (0.00109) & 0.02732 & 0.03805 & 0.03081 \\
8 & 0.04098 & 0.05315 & 0.06387 & 0.04152 & 0.05923 (0.00111) & 0.02784 & 0.03966 & 0.03162 \\
9 & 0.04098 & 0.06228 & 0.06334 & 0.05081 & 0.05959 (0.00138) & 0.03103 & 0.03425 & 0.03661 \\
10 & 0.04142 & 0.05618 & 0.06318 & 0.03630 & 0.05766 (0.00191) & 0.02644 & 0.03264 & 0.03264 \\\hline
